<a href="https://colab.research.google.com/github/LucasAlegre/vote-network/blob/master/vote_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and import libraries

In [1]:
! pip install --user graphistry
! pip install python-igraph

     |████████████████████████████████| 88 kB 694 kB/s 
     |████████████████████████████████| 25.6 MB 21.4 MB/s 
     |████████████████████████████████| 3.3 MB 1.3 MB/s 
  Created wheel for python-igraph: filename=python_igraph-0.9.8-py3-none-any.whl size=9070 sha256=1e8a4c7a2cb968f4753a6997b654e4d72fa7acb4d9e770c53a5a813c8a154602
  Stored in directory: /home/lucas/.cache/pip/wheels/b7/e9/59/1715167232db90e382a3b281ad4a0a1930addd4077ef633232
Successfully built python-igraph


In [1]:
import graphistry
import pandas as pd
import urllib.request
import requests
from datetime import date
import numpy as np
from igraph import Graph, summary
from itertools import combinations
import os
# Init graphistry
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="LucasAlegre", password="")

## Data Retrieval and Processing

In [2]:
start_date = '2019-01-31' #@param {type:"date"}
end_date = '2021-12-30' #@param {type:"date"}
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [3]:
# Collect all votes between start and end dates
for year in range(start_date.year, end_date.year + 1):
    # Read or download votes
    path = 'data/votacoesVotos-{}.csv'.format(year)
    if not os.path.isfile(path) or year == 2021:
        urllib.request.urlretrieve('https://dadosabertos.camara.leg.br/arquivos/votacoesVotos/csv/votacoesVotos-{}.csv'.format(year), path)
    vote_data = pd.read_csv(path, sep=';')

    # Remove simbolic votes
    vote_data = vote_data[vote_data['voto'] != 'Simbólico']

    # Filter dates
    vote_data['dataHoraVoto'] = pd.to_datetime(vote_data['dataHoraVoto'])    
    vote_data = vote_data.loc[(vote_data['dataHoraVoto'] >= start_date) & (vote_data['dataHoraVoto'] <= end_date)]
    vote_data.sort_values(by=['dataHoraVoto'], inplace=True)

    if year == start_date.year:
        votes = vote_data
    else:
        votes = pd.concat([votes, vote_data])
votes

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto
25046,2179189-45,https://dadosabertos.camara.leg.br/api/v2/vota...,2019-02-12 16:09:52,Obstrução,160592,https://dadosabertos.camara.leg.br/api/v2/depu...,Zeca Dirceu,PT,https://dadosabertos.camara.leg.br/api/v2/part...,PR,56.0,http://www.camara.gov.br/internet/deputado/ban...
24899,2179189-45,https://dadosabertos.camara.leg.br/api/v2/vota...,2019-02-12 16:10:00,Não,160674,https://dadosabertos.camara.leg.br/api/v2/depu...,Hugo Motta,PRB,https://dadosabertos.camara.leg.br/api/v2/part...,PB,56.0,http://www.camara.gov.br/internet/deputado/ban...
24981,2179189-45,https://dadosabertos.camara.leg.br/api/v2/vota...,2019-02-12 16:10:02,Não,194260,https://dadosabertos.camara.leg.br/api/v2/depu...,Nivaldo Albuquerque,PTB,https://dadosabertos.camara.leg.br/api/v2/part...,AL,56.0,http://www.camara.gov.br/internet/deputado/ban...
24848,2179189-45,https://dadosabertos.camara.leg.br/api/v2/vota...,2019-02-12 16:10:05,Não,141421,https://dadosabertos.camara.leg.br/api/v2/depu...,Eduardo da Fonte,PP,https://dadosabertos.camara.leg.br/api/v2/part...,PE,56.0,http://www.camara.gov.br/internet/deputado/ban...
24883,2179189-45,https://dadosabertos.camara.leg.br/api/v2/vota...,2019-02-12 16:10:09,Não,74270,https://dadosabertos.camara.leg.br/api/v2/depu...,Gilberto Nascimento,PSC,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56.0,http://www.camara.gov.br/internet/deputado/ban...
...,...,...,...,...,...,...,...,...,...,...,...,...
20623,2203957-126,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-28 16:47:09,Sim,204535,https://dadosabertos.camara.leg.br/api/v2/depu...,Sâmia Bomfim,PSOL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56.0,http://www.camara.gov.br/internet/deputado/ban...
20426,2203957-126,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-28 16:47:32,Não,178977,https://dadosabertos.camara.leg.br/api/v2/depu...,Eduardo Cury,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56.0,http://www.camara.gov.br/internet/deputado/ban...
20419,2203957-126,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-28 16:47:32,Não,204434,https://dadosabertos.camara.leg.br/api/v2/depu...,Dra. Vanda Milani,SOLIDARIEDADE,https://dadosabertos.camara.leg.br/api/v2/part...,AC,56.0,http://www.camara.gov.br/internet/deputado/ban...
20523,2203957-126,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-28 16:48:33,Sim,139285,https://dadosabertos.camara.leg.br/api/v2/depu...,Lídice da Mata,PSB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56.0,http://www.camara.gov.br/internet/deputado/ban...


In [5]:
def get_total_expenses(deputy_id, start_date, end_date):
    #expenses = []
    total = 0.0
    years = '&'.join(['ano={}'.format(year) for year in range(start_date.year, end_date.year+1)])
    url = "https://dadosabertos.camara.leg.br/api/v2/deputados/{}/despesas?{}&itens=100000&ordem=ASC&ordenarPor=ano".format(deputy_id, years)
    read_all = False
    while not read_all:
        read_all = True
        page = requests.get(url).json()
        if 'dados' not in page:
            break
        total += sum([e['valorDocumento'] for e in page['dados']])
        #this_expenses = [{'tipo': e['tipoDespesa'], 'valor': e['valorDocumento']} for e in page['dados']]
        #expenses.extend(this_expenses)
        for link in page['links']:
            if link['rel'] == 'next':
                url = link['href']
                read_all = False
                break
    return total

def get_deputy_info(deputy_id):
    url = 'https://dadosabertos.camara.leg.br/api/v2/deputados/{}'.format(deputy_id)
    data = requests.get(url).json()['dados']
    ultimoStatus = data['ultimoStatus']
    today = date.today()
    nascimento = pd.to_datetime(data['dataNascimento'])
    idade = today.year - nascimento.year - ((today.month, today.day) < (nascimento.month, nascimento.day))
    return {'sexo': data['sexo'], 
            'escolaridade': data['escolaridade'], 
            'idade': idade, 
            'e-mail': ultimoStatus['email'], 
            'situação': ultimoStatus['situacao'], 
            'cidade natal': data['municipioNascimento']}

In [6]:
#%% Take care of different names for same deputy
for group, df_group in votes.groupby('deputado_id'):
    votes['deputado_nome'].loc[votes['deputado_id'] == group] = sorted(df_group['deputado_nome'].unique())[0]

#%% Partidos que mudaram de nome
votes['deputado_siglaPartido'].replace('PMDB', 'MDB', inplace=True)
votes['deputado_siglaPartido'].replace('PRB', 'REPUBLICANOS', inplace=True)
votes['deputado_siglaPartido'].replace('PR', 'PL', inplace=True)
votes['deputado_siglaPartido'].replace('PATRIOTA', 'PATRI', inplace=True)
votes['deputado_siglaPartido'].replace('PPS', 'CIDADANIA', inplace=True)

# Logo dos partidos
party_logo = {}
for p in votes['deputado_uriPartido'].unique():
    if not pd.isna(p):
        dados = requests.get(p).json()['dados']
        if dados['sigla'] == 'MDB':
            party_logo[p] = 'https://logodownload.org/wp-content/uploads/2018/04/mdb-logo-partido.png'
        elif dados['sigla'] == 'SOLIDARIEDADE':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/f/fe/Logomarca_do_Partido_Solidariedade.png'
        elif dados['sigla'] == 'PATRI':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/en/8/8e/Patriota_logo.png'
        elif dados['sigla'] == 'REPUBLICANOS':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/en/0/0d/Republicanos_logo.png'
        elif dados['sigla'] == 'PL':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/0/03/PL-logo.jpg'
        elif dados['sigla'] == 'CIDADANIA':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/d/d7/Logo_do_Cidadania_23.png'
        elif dados['sigla'] == 'NOVO':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/b/b5/Novo30_AOC.png'
        else:
            party_logo[p] = dados['urlLogo']

#all_data['deputado_siglaPartido'].replace('PPL', np.nan, inplace=True) # PPL foi incorporado
#all_data['deputado_siglaPartido'].replace('PRP', np.nan, inplace=True) # PRP foi incorporado
#all_data['deputado_siglaPartido'].replace('PHS', np.nan, inplace=True) # PHS foi incorporado
# all_data = pd.merge(all_data, motions_themes, on="idVotacao", how="inner") 
#all_data['deputado_siglaPartido'].fillna('S.PART.', inplace=True)
# all_data.groupby('idVotacao')['voto'].count()

#all_data.to_csv('votos_{}_to_{}.csv'.format(start_date, end_date), index=False)

/home/lucas/miniconda3/envs/PhD/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## Graph Construction

In [7]:
def generalized_similarity(m, min_eps=0.001, max_iter=1000):
    """ Balázs Kovács, "A generalized model of relational similarity," Social Networks, 32(3), July 2010, pp. 197–211
        Based on: https://github.com/dzinoviev/generalizedsimilarity
    """
    arcs0 = m - m.mean(axis=1)[:, np.newaxis]
    arcs1 = m.T - m.mean(axis=0)[:, np.newaxis]

    eps = min_eps + 1
    N = np.eye(m.shape[1])

    iters = 0
    while (eps > min_eps and iters < max_iter) or np.isnan(N).any():
        M = arcs0.dot(N).dot(arcs0.T)
        m = np.sqrt(M.diagonal())
        M = ((M / (m+1e-8)).T / (m+1e-8)).T
        
        Np = arcs1.dot(M).dot(arcs1.T)
        n = np.sqrt(Np.diagonal())
        Np = ((Np / (n+1e-8)).T / (n+1e-8)).T
        eps = np.abs(Np - N).max()
        N = Np

        iters += 1
    return M
    
def filter_edges(edges_list, num_nodes, threshold=None, density=0.1):
    edges, weights = [], []
    if threshold is not None:
        for e in edges_list:
            if e[1] >= threshold:
                edges.append(e[0])
                weights.append(e[1])
    else:
        count = int(num_nodes * (num_nodes - 1) * density / 2)
        edges_list.sort(reverse=True, key=lambda e: e[1])
        edges_list = edges_list[:count]
        edges = [e[0] for e in edges_list]
        weights = [e[1] for e in edges_list]
    return edges, weights

In [8]:
# Vote Matrix
reps = votes['deputado_nome'].unique()
rep_to_ind = {reps[i]: i for i in range(len(reps))}
motions = votes['idVotacao'].unique()
motion_to_ind = {motions[i]: i for i in range(len(motions))}
parties = [p for p in votes['deputado_siglaPartido'].unique() if pd.notna(p)]

vote_matrix = np.zeros((len(reps), len(motions)))
df_grouped = votes.groupby(['idVotacao', 'deputado_nome'])
for group, df_group in df_grouped:
    voto = df_group['voto'].values[0]
    i = rep_to_ind[group[1]]
    j = motion_to_ind[group[0]]
    if voto == "Sim":
        vote_matrix[i,j] = 1
    elif voto == "Não":
        vote_matrix[i,j] = -1

In [9]:
# IGRAPH
M = generalized_similarity(vote_matrix)
edges = []
for dep1, dep2 in combinations(range(len(reps)), 2):
    if M[dep1,dep2] > 0:
        edges.append(((dep1,dep2), M[dep1,dep2]))

graph = Graph(graph_attrs={'name': 'Câmara dos Deputados'}, directed=False)
graph.add_vertices(reps)
filer_edges, weights = filter_edges(edges, num_nodes=graph.vcount(), threshold=0.9998, density=0.1)
graph.add_edges(filer_edges)
graph.es['weight'] = weights
graph.es['similarity'] = weights
maxw = max(graph.es['weight'])
minw = min(graph.es['weight'])
graph.es['weight'] = [(e - minw) / (maxw - minw) for e in graph.es['weight']] # Normalize weights to [0,1]
graph.vs['community'] = graph.community_leiden(weights='weight', objective_function='modularity', n_iterations=100).membership

graph.delete_edges()
edges, weights = filter_edges(edges, num_nodes=graph.vcount(), threshold=0.0, density=0.1)
graph.add_edges(edges)
graph.es['weight'] = weights
graph.es['similarity'] = weights

info = [votes[votes['deputado_nome']==dep] for dep in graph.vs['name']]
graph.vs['Foto'] = [x['deputado_urlFoto'].values[-1] for x in info]
graph.vs['UF'] = [x['deputado_siglaUf'].values[-1] for x in info]
graph.vs['Partido'] = [x['deputado_siglaPartido'].values[-1] for x in info]
graph.vs['URL'] =  [x['deputado_uri'].values[-1] for x in info]
graph.vs['Partido URI'] =  [x['deputado_uriPartido'].values[-1] for x in info]
#graph.vs['Total Despesas (reais)'] = [get_total_expenses(x['deputado_id'].values[-1], start_date, end_date) for x in info]
for i, x in enumerate(info):
    for k, v in get_deputy_info(x['deputado_id'].values[-1]).items():
        if i == 0:
            graph.vs[k] = ['' for _ in range(len(graph.vs))]
        graph.vs[i][k] = v

summary(graph)

IGRAPH UNW- 575 103536 -- Câmara dos Deputados
+ attr: name (g), Foto (v), Partido (v), Partido URI (v), UF (v), URL (v), cidade natal (v), community (v), e-mail (v), escolaridade (v), idade (v), name (v), sexo (v), situação (v), similarity (e), weight (e)


## Visualization

In [10]:
g = graphistry.bind(source='src', destination='dst', 
                    point_label='name', 
                    point_size='name',
                    edge_weight='similarity', 
                    edge_label='similarity', 
                    edge_size='similarity',
                    point_color='community')
(e_df, n_df) = g.igraph2pandas(graph)
g = g.nodes(n_df).edges(e_df)
g = g.nodes(lambda g: g._nodes.assign(community=g._nodes['community'].astype('int32')))
g = g.addStyle(bg={'color': 'white'}, page={'title': 'Câmara dos Deputados'}) 
g = g.encode_point_icon('Partido URI', categorical_mapping=party_logo, shape='circle')
g = g.settings(url_params={
    'pageTitle': 'Câmara dos Deputados',
    'play': 2000,
    'menu': True, 
    'info': True,
    'strongGravity': True,
    'showArrows': False,
    'pointSize': 5.0,
    'pointsOfInterestMax': 100,
    'edgeInfluence': 1})
g.plot(render=True)